# Self-attention

This notebook contains a mini-introduction to the self-attention mechanism from the *Attention Is All You Need*-paper [[Vaswani et al., 2017](https://arxiv.org/abs/1706.03762)]. It establishes a simple extension of the dot-product variant in [[Luong et al., 2015](https://arxiv.org/abs/1508.04025)]. In particular, we discuss the equation of the **scaled dot-product attention** and its interpretation in a bit more detail. This attention variant is usually written as
$$
\operatorname{attention}(\boldsymbol{Q}, \boldsymbol{K}, \boldsymbol{\boldsymbol{V}}) =
\operatorname{softmax} \left( \frac{\boldsymbol{Q} \boldsymbol{K}^\top}{\sqrt{d_k}} \right) \boldsymbol{V}.
$$

Here, the convention is that the main objects are row vectors organized in matrices. For example, **queries**, **keys** and **values** are the rows of $Q \in \mathbb{R}^{m \times d_k}$, $K \in \mathbb{R}^{n \times d_k}$ and $V \in \mathbb{R}^{n \times d_v}$, respectively. One easily sees that $\boldsymbol{Q} \boldsymbol{K}^\top \in \mathbb{R}^{m \times n}$ is a matrix of query-key dot-products
$$
\boldsymbol{Q} \boldsymbol{K}^\top =
\begin{pmatrix}
  \text{-} & \boldsymbol{q}_1 & \text{-} \\
  \text{-} & \boldsymbol{q}_2 & \text{-} \\
  \vdots & \vdots & \vdots \\
  \text{-} & \boldsymbol{q}_m & \text{-} \\
\end{pmatrix}
\begin{pmatrix}
  \vert & \vert & \ldots & \vert \\
  \boldsymbol{k}_1 & \boldsymbol{k}_2 & \ldots & \boldsymbol{k}_n \\
  \vert & \vert & \ldots & \vert \\
\end{pmatrix} =
\begin{pmatrix}
  \boldsymbol{q}_1 \cdot \boldsymbol{k}_1 & \boldsymbol{q}_1 \cdot \boldsymbol{k}_2 &
  \ldots & \boldsymbol{q}_1 \cdot \boldsymbol{k}_n \\
  \boldsymbol{q}_2 \cdot \boldsymbol{k}_1 & \boldsymbol{q}_2 \cdot \boldsymbol{k}_2 &
  \ldots & \boldsymbol{q}_2 \cdot \boldsymbol{k}_n \\
  \vdots & \vdots & \ddots & \vdots \\
  \boldsymbol{q}_m \cdot \boldsymbol{k}_1 & \boldsymbol{q}_m \cdot \boldsymbol{k}_1 &
  \ldots & \boldsymbol{q}_m \cdot \boldsymbol{k}_n \\
\end{pmatrix}.
$$

Each element of this matrix is divided by $\sqrt{d_k}$. This scaling avoids extremely small gradients for large $d_k$ in the following softmax operation. The softmax is applied over each row separately. Finally, $\boldsymbol{W} = \operatorname{softmax} \left( d_k^{-1/2} \boldsymbol{Q} \boldsymbol{K}^\top \right)$ is right-multiplied by the values. This yields
$$
\operatorname{softmax} \left( \frac{\boldsymbol{Q} \boldsymbol{K}^\top}{\sqrt{d_k}} \right) \boldsymbol{V} =
\begin{pmatrix}
  w_{11} & w_{12} & \ldots & w_{1n} \\
  w_{21} & w_{22} & \ldots & w_{2n} \\
  \vdots & \vdots & \ddots & \vdots \\
  w_{m1} & w_{m2} & \ldots & w_{mn} \\
\end{pmatrix}
\begin{pmatrix}
  \text{-} & \boldsymbol{v}_1 & \text{-} \\
  \text{-} & \boldsymbol{v}_2 & \text{-} \\
  \vdots & \vdots & \vdots \\
  \text{-} & \boldsymbol{v}_n & \text{-} \\
\end{pmatrix} =
\begin{pmatrix}
  \text{-} & \sum_{j=1}^n w_{1j} \boldsymbol{v}_j & \text{-} \\
  \text{-} & \sum_{j=1}^n w_{2j} \boldsymbol{v}_j & \text{-} \\
  \vdots & \vdots & \vdots \\
  \text{-} & \sum_{j=1}^n w_{mj} \boldsymbol{v}_j & \text{-} \\
\end{pmatrix}.
$$

The matrix $d_k^{-1/2} \boldsymbol{Q} \boldsymbol{K}^\top$ is sometimes referred to as **alignment scores**, while $\boldsymbol{W}$ is called the **attention weights**. In each row $i \in \{1,\ldots,m\}$ one has that the weights sum to one $\sum_{j=1}^n w_{ij} = 1$. Hence, $\operatorname{attention}(\boldsymbol{Q}, \boldsymbol{K}, \boldsymbol{\boldsymbol{V}})  \in \mathbb{R}^{m \times d_v}$ can be seen as a relevance-weighted average of the values.

Let us further investigate a single row $(w_{i1}, \ldots, w_{in})$ of the attention weight matrix. It describes how the query vector $\boldsymbol{q}_i$ is matched against (or attends to) all possible keys $\boldsymbol{k}_j$ with $j=1,\ldots,n$. This highlights the analogy to information retrieval. The row vector of attention weights is given as
$$
(w_{i1}, \ldots, w_{in}) = \operatorname{softmax} \left(
\frac{\boldsymbol{q}_i \cdot \boldsymbol{k}_1}{\sqrt{d_k}}, \ldots,
\frac{\boldsymbol{q}_i \cdot \boldsymbol{k}_n}{\sqrt{d_k}} \right) =
\frac{\left( \exp \left( \frac{\boldsymbol{q}_i \cdot \boldsymbol{k}_1}{\sqrt{d_k}} \right), \ldots,
\exp \left( \frac{\boldsymbol{q}_i \cdot \boldsymbol{k}_n}{\sqrt{d_k}} \right) \right)}
{\sum_{j=1}^n \exp \left( \frac{\boldsymbol{q}_i \cdot \boldsymbol{k}_j}{\sqrt{d_k}} \right)}.
$$

The attention above is defined by three weight matrices $\boldsymbol{W}_q \in \mathbb{R}^{d_x \times d_k}$, $\boldsymbol{W}_k \in \mathbb{R}^{d_x \times d_k}$ and $\boldsymbol{W}_v \in \mathbb{R}^{d_x \times d_v}$. They are used to compute the queries, keys and values. For example, for a single input $\boldsymbol{x} \in \mathbb{R}^{1 \times d_x}$, they would be respectively given as $\boldsymbol{q} = \boldsymbol{x} \boldsymbol{W}_q$, $\boldsymbol{k} = \boldsymbol{x} \boldsymbol{W}_k$ and $\boldsymbol{v} = \boldsymbol{x} \boldsymbol{W}_v$. For multiple inputs $\boldsymbol{X}$ one can similarly write
$$
\boldsymbol{Q} = \boldsymbol{X} \boldsymbol{W}_q, \quad
\boldsymbol{K} = \boldsymbol{X} \boldsymbol{W}_k, \quad
\boldsymbol{V} = \boldsymbol{X} \boldsymbol{W}_v.
$$

When queries, keys, and values are computed for a single sequence, say its elements are stacked in row-wise fashion so as to construct $\boldsymbol{X}$, then one speaks of **self-attention**. It relates the items from a single sequence to each other. More generally, one may relate items from two different sequences.

A common extension to the mechanism presented above is **multi-head attention**. Here, one simply uses multiple independent attentions with different weight matrices. Their outputs are eventually concatenated.

It has to be noted that the attention mechanism ignores any sequential structure altogether. Thus one has to rely on a so-called **positional encoding** in order to provide and process order information. Such an encoding $\boldsymbol{P}$ is often constructed such that it can be added to the token embeddings $\boldsymbol{X}$. Then $\boldsymbol{X} + \boldsymbol{P}$ is used in all downstream operations that follow.